In [2]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import gdown
import os
import csv
import time

In [ ]:
cap = cv.VideoCapture(0)

In [ ]:
ret, frame = cap.read()

In [ ]:
plt.imshow(frame)

In [ ]:
cap.release()

In [3]:
# Pfad zur XML-Datei relativ zum Skriptverzeichnis
xml_file_path = 'haarcascade_frontalface_default.xml'

# Überprüfen, ob die XML-Datei bereits vorhanden ist
if not os.path.exists(xml_file_path):
    # Wenn die Datei nicht vorhanden ist, laden Sie sie von Google Drive herunter
    gdown.download('https://drive.google.com/uc?id=1N5j5ke98qCt_0J70wg6F8diHrF5qqxeX&export=download', xml_file_path, quiet=False)
    print("XML file downloaded successfully.")
else:
    print("XML file already exists.")

cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()
    
    # Laden der Gesichtserkennungsklassifikatoren
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

# Create folder to store faces if it doesn't exist
objects_folder = 'objects'
if not os.path.exists(objects_folder):
    os.mkdir(objects_folder)
folder = "test"
person_folder = os.path.join(objects_folder, folder)
if not os.path.exists(person_folder):
    os.mkdir(person_folder)

# Variables for saving mechanism
frame_count = 0
save_frame = True

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # Convert frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw rectangle around faces and save images
    for (x, y, w, h) in faces:
        cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        if save_frame:
            cv.imwrite(os.path.join(person_folder, f"frame_{frame_count}.png"), frame)
            with open(os.path.join(person_folder, f"frame_{frame_count}.csv"), "w", newline="") as csvfile:
                writer = csv.writer(csvfile, delimiter=",")
                writer.writerow([x, y, w, h])
            frame_count += 1
            if frame_count >= 30:
                save_frame = False

    # Show frame with faces
    cv.imshow('frame', frame)

    # Check for 'q' key to quit
    if cv.waitKey(1) == ord('q'):
        break

# Release video capture and close all windows
cap.release()
cv.destroyAllWindows()

XML file already exists.


In [18]:
import cv2 as cv
import os
import csv
import random

from common import TRAIN_FOLDER, VAL_FOLDER, ROOT_FOLDER

# Funktion zum Entfernen aller Dateien in einem Ordner
def clean_folder(folder):
    for root, dirs, files in os.walk(folder):
        for file in files:
            os.remove(os.path.join(root, file))

# Funktion zum Erstellen von Trainings- und Validierungsordnern
def create_folders():
    for folder in [TRAIN_FOLDER, VAL_FOLDER]:
        if not os.path.exists(folder):
            os.mkdir(folder)

# Funktion zum Ausschneiden der Gesichter
def crop_faces(border, split):
    # Sicherstellen, dass die Ordner existieren und leer sind
    clean_folder(TRAIN_FOLDER)
    clean_folder(VAL_FOLDER)
    create_folders()

    # Ordner von Personen durchsuchen
    for root, dirs, files in os.walk(ROOT_FOLDER):
        for folder_name in dirs:
            person_folder = os.path.join(root, folder_name)

            # Liste der Bilddateien im aktuellen Ordner erhalten
            image_files = [file for file in os.listdir(person_folder) if file.endswith('.png')]

            for image_file in image_files:
                # Dateipfad zum aktuellen Bild
                image_path = os.path.join(person_folder, image_file)

                # CSV-Dateipfad zum aktuellen Bild
                csv_file_path = os.path.join(person_folder, f"{os.path.splitext(image_file)[0]}.csv")

                # Sicherstellen, dass die CSV-Datei existiert
                if not os.path.exists(csv_file_path):
                    continue

                # Lesen des Bildes
                frame = cv.imread(image_path)

                # Lesen der CSV-Datei und Extrahieren der Gesichtskoordinaten
                with open(csv_file_path, 'r') as csv_file:
                    csv_reader = csv.reader(csv_file)
                    for row in csv_reader:
                        # Gesichtskoordinaten extrahieren (Annahme: x, y, width, height)
                        x, y, w, h = map(int, row)

                        # Gesicht ausschneiden
                        face = frame[y:y+h, x:x+w]

                        # Hier die Logik für das Hinzufügen des Randes implementieren
                        border_pixels = int(min(face.shape[0], face.shape[1]) * border)
                        face_with_border = cv.copyMakeBorder(face, border_pixels, border_pixels, border_pixels, border_pixels, cv.BORDER_REFLECT)

                        # Entscheiden, ob das Bild dem Trainings- oder Validierungsordner zugewiesen wird
                        if random.uniform(0.0, 1.0) < split:
                            save_folder = VAL_FOLDER
                        else:
                            save_folder = TRAIN_FOLDER

                        # Bild speichern
                        cv.imwrite(os.path.join(save_folder, f"{folder_name}_{image_file}"), face_with_border)

                        print(f"Face cropped from {image_file} and saved to {save_folder}")

# Beispiel für die Verwendung der Funktion crop_faces
border = 0.01  # Randgröße als prozentualer Faktor
split = 0.3   # Anteil der Bilder, die im Validierungsordner landen sollen
crop_faces(border, split)

Face cropped from frame_1b9af5f9-f124-46f5-8327-c272dc6cb792.png and saved to val
Face cropped from frame_259501b2-f490-4800-a76a-a975e27097cd.png and saved to train
Face cropped from frame_30bbd8b9-748c-4984-bf71-1129d636f4d3.png and saved to train
Face cropped from frame_5760e273-0a83-461b-a95a-3f89d710070e.png and saved to train
Face cropped from frame_5d9eb006-51c8-4ca8-9c52-f87c3ae3c166.png and saved to val
Face cropped from frame_6157ca56-e93d-43df-ab70-901436bf7cce.png and saved to train
Face cropped from frame_920cc1c4-c0ee-48d9-b8c2-30658fc02aa5.png and saved to train
Face cropped from frame_951fbced-386d-4e97-b5d2-48f3f6540753.png and saved to train
Face cropped from frame_9e9ad275-8135-4789-a9ec-21a33234cf9f.png and saved to train
Face cropped from frame_a3cf08cd-dc12-4334-aa5b-464a7bfb4325.png and saved to val
Face cropped from frame_b34f4c6b-90fd-4bc9-9756-1aa325ac8c69.png and saved to train
Face cropped from frame_bfefddde-425c-4dcf-8199-9c6e1eceaab5.png and saved to val
